In [1]:
import sys
print(sys.prefix)
!git clone https://github.com/zhouhaoyi/Informer2020.git
!git clone https://github.com/zhouhaoyi/ETDataset.git
!ls

/home/ailabguest/anaconda3/envs/babble
fatal: destination path 'Informer2020' already exists and is not an empty directory.
fatal: destination path 'ETDataset' already exists and is not an empty directory.
create-csv.ipynb      informer.ipynb		    output-with-date.csv
ETDataset	      output.csv		    results
Informer2020	      output-with-date2.csv	    time-epoc.py
informer_checkpoints  output-with-date-average.csv


In [2]:
import sys
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

In [4]:

%pip install -r ./Informer2020/requirements.txt

     |████████████████████████████████| 13.1 MB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 10.5 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.7 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 735.5 MB 5.4 kB/s  eta 0:00:01     |█████▉                          | 135.3 MB 12.1 MB/s eta 0:00:50     |███████████▊                    | 270.5 MB 10.7 MB/s eta 0:00:44     |██████████████████████████▏     | 600.9 MB 12.1 MB/s eta 0:00:12
  Attempting uninstall: numpy
    Found existing installation: numpy 1.17.5
    Uninstalling numpy-1.17.5:
      Successfully uninstalled numpy-1.17.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.0
    Uninstalling pandas-1.0.0:
      Successf

In [3]:

from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch


In [13]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'custom' # data
args.root_path = './' # root path of data file
args.data_path = 'output-with-date.csv' # data file
args.features = 'S' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'pressure' # target feature in S or MS task
args.freq = 's' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.seq_len = 48#mod 96 # input sequence length of Informer encoder
args.label_len = 24#mod 48 # start token length of Informer decoder
args.pred_len = 12#mod 24 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 1 # encoder input size
args.dec_in = 1 # decoder input size
args.c_out = 1 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 256#modified # dimension of model
args.n_heads = 1#modified 8 # num of heads
args.e_layers = 1#modified 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 1024#modified 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0

args.batch_size = 16 #modified 32
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 3#modified
args.patience = 1#modified
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'


In [14]:
import torch.cuda

print(torch.cuda.is_available())

False


In [15]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

In [16]:
# Set augments by using data name
data_parser = {
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

In [17]:
args.detail_freq = args.freq
args.freq = args.freq[-1:]

In [18]:
print('Args in experiment:')
print(args)
print(f"Max epochs: {args.train_epochs}")


Args in experiment:
{'model': 'informer', 'data': 'custom', 'root_path': './', 'data_path': 'output-with-date.csv', 'features': 'S', 'target': 'pressure', 'freq': 's', 'checkpoints': './informer_checkpoints', 'seq_len': 48, 'label_len': 24, 'pred_len': 12, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'factor': 5, 'd_model': 256, 'n_heads': 1, 'e_layers': 1, 'd_layers': 1, 'd_ff': 1024, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'batch_size': 16, 'learning_rate': 0.0001, 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 1, 'train_epochs': 3, 'patience': 1, 'des': 'exp', 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'detail_freq': 's'}
Max epochs: 3


In [19]:
Exp = Exp_Informer

In [ ]:
for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

    # set experiments
    exp = Exp(args)
    
    # train
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)
    
    # test
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)

    torch.cuda.empty_cache()
    

Use CPU
>>>>>>>start training : informer_custom_ftS_sl48_ll24_pl12_dm256_nh1_el1_dl1_df1024_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 85957
val 12277
test 24565
	iters: 100, epoch: 1 | loss: 1.0268863
	speed: 0.1141s/iter; left time: 1826.7800s
	iters: 200, epoch: 1 | loss: 0.9277111
	speed: 0.1158s/iter; left time: 1843.2594s
	iters: 300, epoch: 1 | loss: 1.0305418
	speed: 0.1164s/iter; left time: 1840.9387s
	iters: 400, epoch: 1 | loss: 0.9032533
	speed: 0.1158s/iter; left time: 1820.6007s
	iters: 500, epoch: 1 | loss: 1.0470827
	speed: 0.1162s/iter; left time: 1815.1579s
	iters: 600, epoch: 1 | loss: 1.0090394
	speed: 0.1163s/iter; left time: 1804.4953s
	iters: 700, epoch: 1 | loss: 0.9071641
	speed: 0.1153s/iter; left time: 1777.2423s
	iters: 800, epoch: 1 | loss: 0.9319744
	speed: 0.1157s/iter; left time: 1771.7945s
	iters: 900, epoch: 1 | loss: 1.1203487
	speed: 0.1160s/iter; left time: 1765.9133s
	iters: 1000, epoch: 1 | loss: 1.0979403
	speed: 0.117

In [ ]:
from data.data_loader import Dataset_Custom
from torch.utils.data import DataLoader
import pandas as pd
import os

# args.target = 'HULL'
# args.freq = 'h'

Data = Dataset_Custom
timeenc = 0 if args.embed!='timeF' else 1
flag = 'test'; shuffle_flag = False; drop_last = True; batch_size = 1

data_set = Data(
    root_path=args.root_path,
    data_path=args.data_path,
    flag=flag,
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    timeenc=timeenc,
    target=args.target, # HULL here
    freq=args.freq # 'h': hourly, 't':minutely
)
data_loader = DataLoader(
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    drop_last=drop_last)